In [1]:
import os
import cv2
import json
import shutil

import numpy as np

from glob import glob


c:\Users\frank\Anaconda3\envs\pytorch\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\frank\Anaconda3\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\frank\Anaconda3\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\Users\frank\Anaconda3\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Seperating images and labels for AI+ data

In [6]:
data_path = "C:/Users/frank/OneDrive/Desktop/GT_Practicum/data/AI+/Ultrasound-labeled/malignant/"

img_list = glob(os.path.join(data_path, '*/*.jpg'))
json_list = glob(os.path.join(data_path, '*/*.json'))

In [7]:
print("Number of images: {}".format(len(img_list)))
print("Number of jsons: {}".format(len(json_list)))

assert len(img_list) == len(json_list)

Number of images: 488
Number of jsons: 488


In [4]:
img_des_path = "C:/Users/frank/OneDrive/Desktop/GT_Practicum/data/preprocessed/AI+/malignant/img/"
json_des_path = "C:/Users/frank/OneDrive/Desktop/GT_Practicum/data/preprocessed/AI+/malignant/json/"

In [5]:
img_list.sort()
json_list.sort()

for img_path, json_path in zip(img_list, json_list):
    assert os.path.basename(img_path)[:-4] == os.path.basename(json_path)[:-5]
    img_des = os.path.join(img_des_path, os.path.basename(img_path))
    json_des = os.path.join(json_des_path, os.path.basename(json_path))
    shutil.copyfile(img_path, img_des)
    shutil.copyfile(json_path, json_des)

# Decide Crop Size for Each Batch

Before proceeding, need to manually divide the images to different batches based on their device brand (e.g., SonoScape).

As images from different brand of scanners have different GUI layouts.

Then crop images from each brand using same setups.

In [ ]:
data_path = "C:/Users/frank/OneDrive/Desktop/GT_Practicum/data/preprocessed/AI+/malignant/"

img_list = glob(os.path.join(data_path, './img/SonoScape/*.jpg'))
json_folder = "C:/Users/frank/OneDrive/Desktop/GT_Practicum/data/preprocessed/AI+/malignant/json/"

assert len(img_list) == len(json_list)

In [102]:
img_list.sort()
# json_list.sort()

dis_upper = []
dis_lower = []
dis_left = []
dis_right = []

# Measeure Distances of tumor rectangular box to image edges

for img_path in img_list:
    img = cv2.imread(img_path)
    h, w, _ = img.shape
    json_path = os.path.join(json_folder, os.path.basename(img_path)[:-4] + '.json')
    with open(json_path, 'r') as f:
        data = json.load(f)
        for i in range(len(data['shapes'])):
            dis_upper.append(data['shapes'][i]['points'][0][1])
            dis_lower.append(h - data['shapes'][i]['points'][1][1])
            dis_left.append(data['shapes'][i]['points'][0][0])
            dis_right.append(w - data['shapes'][i]['points'][1][0])

print(len(dis_upper))
assert len(dis_upper) == len(dis_lower) == len(dis_left) == len(dis_right)

In [ ]:
print(f"Minimum Distances to Crop from top for the image batch: {np.min(dis_upper)}")
print(f"Minimum Distances to Crop from bottom for the image batch: {np.min(dis_lower)}")
print(f"Minimum Distances to Crop from left for the image batch: {np.min(dis_left)}")
print(f"Minimum Distances to Crop from right for the image batch: {np.min(dis_right)}")

# Crop Images by Batches

In [109]:
new_data_path = "C:/Users/frank/OneDrive/Desktop/GT_Practicum/data/preprocessed/AI+/malignant/crop_img/SonoScape/"

for img_path in img_list:
    img = cv2.imread(img_path)
    img_crop = img[110:-180, 330:-210, :]
    # img_crop = img
    new_img_path = os.path.join(new_data_path, os.path.basename(img_path))
    cv2.imwrite(new_img_path, img_crop)

    json_path = os.path.join(json_folder, os.path.basename(img_path)[:-4] + '.json')
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    for i in range(len(data['shapes'])):
        data['shapes'][i]['label'] = 'malignant'
        data['shapes'][i]['points'][0][0] = data['shapes'][i]['points'][0][0] - 330
        data['shapes'][i]['points'][1][0] = data['shapes'][i]['points'][1][0] - 330
        data['shapes'][i]['points'][0][1] = data['shapes'][i]['points'][0][1] - 110
        data['shapes'][i]['points'][1][1] = data['shapes'][i]['points'][1][1] - 110

    data['imageHeight'] = img_crop.shape[0]
    data['imageWidth'] = img_crop.shape[1]
    data['imageData'] = None


    new_json_path = os.path.join(new_data_path, os.path.basename(img_path)[:-4] + '.json')
    with open(new_json_path, 'w') as f:
        json.dump(data, f)
    